In [1]:
from tqdm import tqdm
from glob import glob

import copy
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
import os
import seaborn as sb
import matplotlib.pyplot as plt
import collections

from makiflow.augmentation import AffineAugment, ElasticAugment, ImageCutter, Data
from makiflow.augmentation.segmentation import HCScanner
from makiflow.augmentation.segmentation import GDBalancer
from makiflow.augmentation.segmentation.balancing.utils import hcv_to_num
from makiflow.augmentation.segmentation import GD2BBuilder
from makiflow.tf_scripts import set_main_gpu
set_main_gpu(0)

In [2]:
def mutate_masks(masks, mapping):
    """
    Remaps classes on the given `masks` according to the `mapping`.
    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    mapping : list
        List of tuples: [(source_class_number, new_class_number)],
        where `source_class_number` will be changed to `new_class_number` in the `masks`.
    Returns
    ---------
    new_masks : the same type as `masks`
        New masks with changed class numbers.
    """
    if type(mapping) is not list or (len(mapping) != 0 and type(mapping[0]) is not tuple):
        raise TypeError('mapping should be list of typles')

    new_masks = copy.deepcopy(masks)

    for i in range(len(new_masks)):
        for elem in mapping:
            old_value = elem[0]
            new_value = elem[1]
            new_masks[i][masks[i] == old_value] = new_value

    return  new_masks

In [3]:
set_main_gpu(0)

In [483]:
batch_num = 3

In [484]:
mask_names = glob(f'/mnt/data/med_data/balanced_batches/batch_{batch_num}/train_set/origin_wo_4_set/masks/*.bmp')

In [485]:
len(mask_names)

99

In [486]:
masks = [cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE) for mask_name in mask_names]

In [487]:
masks = {mask_name: mask for mask_name, mask in zip(mask_names, masks)}

In [488]:
scanner = HCScanner(masks, 9)
scanner.scan()
scanner.save_info(f'uniq_hvc_{batch_num}.csv', f'masks_hcvg_{batch_num}.csv')
len(scanner.hcv_groups)

Saved!


13

In [489]:
masks_hcvg = pd.read_csv(f'masks_hcvg_{batch_num}.csv', index_col=False)
masks_hcvg.columns = ['path', 'hcvg']
masks_hcvg.head()

,path,hcvg
0,/mnt/data/med_data/balanced_batches/batch_3/tr...,319
1,/mnt/data/med_data/balanced_batches/batch_3/tr...,319
2,/mnt/data/med_data/balanced_batches/batch_3/tr...,447
3,/mnt/data/med_data/balanced_batches/batch_3/tr...,319
4,/mnt/data/med_data/balanced_batches/batch_3/tr...,271


In [490]:
for index, row in masks_hcvg.iterrows():
    print(f"Path to image: {row['path']}, hcvg: {row['hcvg']}")
    break

Path to image: /mnt/data/med_data/balanced_batches/batch_3/train_set/origin_wo_4_set/masks/0.bmp, hcvg: 319


In [491]:
uniq_hvc = pd.read_csv(f'uniq_hvc_{batch_num}.csv', index_col=False)
# Be carefull, here is unpacking
uniq_hvc.columns = list(['hcvg', *uniq_hvc.columns[1:]])
uniq_hvc.head()

,hcvg,0,1,2,3,4,5,6,7,8
0,319,1,1,1,1,1,1,0,0,1
1,447,1,1,1,1,1,1,0,1,1
2,271,1,1,1,1,0,0,0,0,1
3,303,1,1,1,1,0,1,0,0,1
4,287,1,1,1,1,1,0,0,0,1


In [492]:
uniq_num = uniq_hvc.get_values()
uniq_num = sorted(uniq_num, key=lambda x: x[0]) 
hcv_groups = [vec[1:] for vec in uniq_num]
# hcv_groups = np.delete(hcv_groups, 4, axis=1)
hcv_groups

[array([1, 1, 1, 1, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 1]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 1]),
 array([1, 1, 1, 1, 0, 1, 0, 0, 1]),
 array([1, 1, 0, 1, 1, 1, 0, 0, 1]),
 array([1, 1, 1, 1, 1, 1, 0, 0, 1]),
 array([1, 1, 1, 1, 1, 0, 1, 0, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 1, 0, 0, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 0, 1, 1]),
 array([1, 1, 1, 1, 1, 0, 1, 1, 1])]

Delete 4 class

## GDBalancer Part

In [493]:
initial_weights = collections.OrderedDict(sorted(scanner.hcv_groups.items()))
initial_weights

OrderedDict([(15, 3),
             (31, 1),
             (271, 5),
             (287, 45),
             (303, 1),
             (315, 2),
             (319, 28),
             (351, 3),
             (379, 1),
             (383, 1),
             (415, 5),
             (447, 3),
             (479, 1)])

In [494]:
initial_weights = np.array(list(initial_weights.values()))

In [495]:
initial_weights

array([ 3,  1,  5, 45,  1,  2, 28,  3,  1,  1,  5,  3,  1])

In [496]:
# Initial cardinality distribution
np.round(initial_weights / initial_weights.sum(), 2).reshape(-1, 1)

array([[0.03],
       [0.01],
       [0.05],
       [0.45],
       [0.01],
       [0.02],
       [0.28],
       [0.03],
       [0.01],
       [0.01],
       [0.05],
       [0.03],
       [0.01]])

In [526]:
#init_w = np.array([80, 40, 70, 290, 40, 50, 250, 70, 40, 50, 100, 40, 50]) * 2 # 1 batch

init_w = np.array([10] * 3 + [300] + [40] *2 + [160] + [10]*6)
#init_w = np.array([20] * 13)
# init_w = np.array([98, 98, 95, 60, 90, 98, 96, 70, 94, 94, 98, 96, 98, 98, 98])
#init_w = np.array([60, 60, 63, 98, 68, 60, 62, 87, 64, 64, 60, 66, 60]) # The best. i think
# init_w = np.array([100, 100, 103, 138, 108, 100, 102, 127, 104, 104, 100, 106, 100, 100, 100])
# init_w = np.array([138, 138, 135, 100, 130, 138, 137, 110, 134, 134, 138, 132, 138, 138, 138])

In [543]:
#init_w = np.array([80, 40, 70, 290, 40, 50, 250, 70, 40, 50, 100, 40, 50]) * 2 
# init_w = np.array([60, 60, 63, 98, 68, 60, 62, 87, 64, 64, 60, 66, 60])
dest_v = np.array([0.99, 0.99, 0.99, 0.99, 0.99, 0.4, 0.2, 0.2, 0.99])

balancer = GDBalancer(
    hcv_groups=np.array(hcv_groups), 
    initial_c=init_w,
    objective='alpha', min_c=20, max_c=2000) # 'uniq_hvc.csv'
optimizer = tf.train.AdamOptimizer(1)

balancer.add_reg2(0.000001, initial_weights)

In [544]:
sess = tf.Session()
balancer.set_session(sess)

In [545]:
balancer.get_percentage()

array([[100.],
       [100.],
       [ 92.],
       [100.],
       [ 90.],
       [ 43.],
       [  6.],
       [  5.],
       [ 97.]], dtype=float32)

In [561]:
iterations = 3
balancer.optimize(dest_v, optimizer, iterations=iterations, print_period=iterations - 1)

0.08150501
[[100.]
 [100.]
 [ 95.]
 [100.]
 [ 93.]
 [ 41.]
 [ 20.]
 [ 20.]
 [ 95.]]
0.08062024
[[100.]
 [100.]
 [ 95.]
 [100.]
 [ 93.]
 [ 41.]
 [ 21.]
 [ 20.]
 [ 95.]]


In [562]:
# Initial cardinality distribution
np.concatenate([np.round(balancer.show_deviation() * -1, 0) ,
                np.round(initial_weights / initial_weights.sum(), 2).reshape(-1, 1)*100], axis=1).astype(np.int32)

array([[ -1,   3],
       [  1,   1],
       [ -3,   5],
       [-11,  45],
       [  1,   1],
       [  0,   2],
       [ -8,  28],
       [  3,   3],
       [  1,   1],
       [  5,   1],
       [  2,   5],
       [  4,   3],
       [  5,   1]], dtype=int32)

In [563]:
balancer.get_weights()

array([[ 20.     ],
       [ 20.     ],
       [ 20.     ],
       [296.28857],
       [ 20.     ],
       [ 20.     ],
       [179.83429],
       [ 49.86645],
       [ 20.     ],
       [ 54.32784],
       [ 57.8166 ],
       [ 59.72967],
       [ 54.6839 ]], dtype=float32)

In [564]:
balancer.get_percentage()

array([[100.],
       [100.],
       [ 95.],
       [100.],
       [ 93.],
       [ 41.],
       [ 21.],
       [ 20.],
       [ 95.]], dtype=float32)

In [565]:
balancer.get_weights().sum()

872.5473

## Prepare values to GD2BBuilder

In [566]:
balancer_weights = balancer.get_weights()
balancer_weights = balancer_weights.flatten().round().astype(np.int32)
balancer_weights

array([ 20,  20,  20, 296,  20,  20, 180,  50,  20,  54,  58,  60,  55],
      dtype=int32)

In [567]:
config = {}
for i in range(len(balancer_weights)):
    config[hcv_to_num(balancer.hcv_groups[i])] = balancer_weights[i]

In [568]:
config

{15: 20,
 31: 20,
 271: 20,
 287: 296,
 303: 20,
 315: 20,
 319: 180,
 351: 50,
 379: 20,
 383: 54,
 415: 58,
 447: 60,
 479: 55}

In [569]:
balancer.save_cardinalities(f'balancer_cardinalities_{batch_num}.csv')

## GD2BBuilder part

In [570]:
lbl_names = glob(f'/mnt/data/med_data/balanced_batches/batch_{batch_num}/train_set/origin_wo_4_set/masks/*.bmp')

In [571]:
mask2image = {}
for mask_name in lbl_names:
    mask2image[mask_name] = mask_name.replace('masks', 'images')

In [572]:
pd.DataFrame.from_dict(mask2image, orient='index', columns=['image']).to_csv(f'mi_{batch_num}.csv')

In [573]:
builder = GD2BBuilder(
    path_to_balance_config=f'balancer_cardinalities_{batch_num}.csv',
    path_to_hc_list=f'masks_hcvg_{batch_num}.csv',
    path_to_mi=f'mi_{batch_num}.csv',
    resize=(1024, 1024)
)

/home/rustam/anaconda3/lib/python3.6/site-packages/makiflow/augmentation/segmentation/balancing/gdbb_builder.py:24: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  self._hc_list = pd.DataFrame.from_csv(path_to_hc_list)
/home/rustam/anaconda3/lib/python3.6/site-packages/makiflow/augmentation/segmentation/balancing/gdbb_builder.py:25: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  self._balance_c = pd.DataFrame.from_csv(path_to_balance_config)
/home/rustam/anaconda3/lib/python3.6/site-packages/makiflow/augmentation/segmentation/balancing/gdbb_builder.py:34: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default ar

Loading masks and images.
Finished.
Group masks and images by their ids.
319 cardinality is 28
447 cardinality is 3
271 cardinality is 5
303 cardinality is 1
287 cardinality is 45
351 cardinality is 3
415 cardinality is 5
315 cardinality is 2
15 cardinality is 3
379 cardinality is 1
479 cardinality is 1
31 cardinality is 1
383 cardinality is 1
Finished.


In [574]:
builder.set_elastic_aug_params((1024, 1024, 3), 700, 11, 7)

In [575]:
builder.create_batch(f'/mnt/data/med_data/balanced_batches/batch_{batch_num}/train_set/aug_set/set_1024_10k_wo_4_4/origin_batch')

Balancing group 319...
Augmentor updated 6 times.
Finished.
319 ready
Balancing group 447...
Augmentor updated 20 times.
Finished.
447 ready
Balancing group 271...
Augmentor updated 4 times.
Finished.
271 ready
Balancing group 303...
Augmentor updated 20 times.
Finished.
303 ready
Balancing group 287...
Augmentor updated 6 times.
Finished.
287 ready
Balancing group 351...
Augmentor updated 16 times.
Finished.
351 ready
Balancing group 415...
Augmentor updated 11 times.
Finished.
415 ready
Balancing group 315...
Augmentor updated 10 times.
Finished.
315 ready
Balancing group 15...
Augmentor updated 6 times.
Finished.
15 ready
Balancing group 379...
Augmentor updated 20 times.
Finished.
379 ready
Balancing group 479...
Augmentor updated 55 times.
Finished.
479 ready
Balancing group 31...
Augmentor updated 20 times.
Finished.
31 ready
Balancing group 383...
Augmentor updated 54 times.
Finished.
383 ready


In [219]:
test = glob(f'/mnt/data/med_data/balanced_batches/batch_{batch_num}/train_set/aug_set/set_1024_wo_elastic_paper_test#3_3/masks/*.bmp')
lbls = []
for mask_name in test:
    lbls += [cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)]
len(lbls)

0

In [ ]:
sb.heatmap(lbls[231])

In [ ]:
plt.imshow(lbls[44])

In [ ]:
import math
def get_CR_vector(lbls, count_classes):
  uniq = dict([(str(i),0) for i in range(count_classes)])
  
  for i in range(len(lbls)):
    a,b = np.unique(lbls[i], return_counts=True)
    for num in a:
        uniq[str(num)] += 1
  print(uniq)
  return [round(uniq[k] / len(lbls),2) for k in uniq]

In [ ]:
get_CR_vector(lbls, 10)

In [ ]:
lbls = {mask_name: mask for mask_name, mask in zip(test, lbls)}
scanner = HCScanner(lbls, 10)
scanner.scan()

In [ ]:
scanner.hcv_groups

In [ ]:
len(lbls)

In [ ]:
def get_train_data(path='/mnt/data/med_data/unbalanced_batches/batch_3'):
    Ytrain = []
    masks = glob.glob(path + '/masks/*.bmp')
    masks.sort()
    for mask_name in tqdm(masks):
        mask = cv2.imread(mask_name)
        Ytrain.append(mask[:,:,0])
        if np.max(mask) >= 10:
            print(np.max(mask), f' in image {mask_name} ')
    return Ytrain

In [ ]:
x = get_train_data()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(Ytrain[0])

In [ ]:
cv2.imwrite('2.bmp', test)

In [ ]:
x=cv2.imread('/mnt/data/med_data/balanced_batches/bb_danil1/masks/527_0.bmp')

In [ ]:
sns.heatmap(x)

In [ ]:
test = cv2.resize(x, (1024,1024), interpolation=cv2.INTER_NEAREST)